Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (10000, 784), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic Regression (using stochastic gradient descent) with L2 regularization

In [5]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    beta = 0.001
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    new_loss = loss + beta*(tf.nn.l2_loss(weights))
   
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.694168
Minibatch accuracy: 10.9%
Validation accuracy: 9.7%
Minibatch loss at step 500: 1.607517
Minibatch accuracy: 78.9%
Validation accuracy: 56.1%
Minibatch loss at step 1000: 0.787669
Minibatch accuracy: 80.5%
Validation accuracy: 59.0%
Minibatch loss at step 1500: 0.571280
Minibatch accuracy: 86.7%
Validation accuracy: 61.2%
Minibatch loss at step 2000: 0.650291
Minibatch accuracy: 78.9%
Validation accuracy: 62.6%
Minibatch loss at step 2500: 0.686348
Minibatch accuracy: 85.2%
Validation accuracy: 63.4%
Minibatch loss at step 3000: 0.561311
Minibatch accuracy: 82.8%
Validation accuracy: 64.2%
Test accuracy: 88.5%


1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes with L2 regularization

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    #Hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    beta = 0.0001
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    new_loss = loss + beta*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(valid_relu, weights)+ biases))
    test_relu = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(test_relu, weights)+ biases))


In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 335.210876
Minibatch accuracy: 11.7%
Validation accuracy: 24.4%
Minibatch loss at step 500: 16.467730
Minibatch accuracy: 82.0%
Validation accuracy: 61.3%
Minibatch loss at step 1000: 9.074248
Minibatch accuracy: 83.6%
Validation accuracy: 63.8%
Minibatch loss at step 1500: 5.619035
Minibatch accuracy: 86.7%
Validation accuracy: 63.8%
Minibatch loss at step 2000: 2.709820
Minibatch accuracy: 83.6%
Validation accuracy: 62.8%
Minibatch loss at step 2500: 8.305853
Minibatch accuracy: 79.7%
Validation accuracy: 63.8%
Minibatch loss at step 3000: 4.129192
Minibatch accuracy: 86.7%
Validation accuracy: 64.4%
Test accuracy: 89.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 10

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 361.134766
Minibatch accuracy: 8.6%
Validation accuracy: 20.1%
Test accuracy: 76.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    #Hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    #Add droupout
    hidden_layer_drop = tf.nn.dropout(hidden_layer,0.5)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    beta = 0.001
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    new_loss = loss + beta*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(valid_relu, weights)+ biases))
    test_relu = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(test_relu, weights)+ biases))


In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 445.304626
Minibatch accuracy: 10.2%
Validation accuracy: 19.2%
Minibatch loss at step 500: 36.162697
Minibatch accuracy: 76.6%
Validation accuracy: 60.1%
Minibatch loss at step 1000: 19.337069
Minibatch accuracy: 80.5%
Validation accuracy: 63.1%
Minibatch loss at step 1500: 12.889247
Minibatch accuracy: 69.5%
Validation accuracy: 59.7%
Minibatch loss at step 2000: 6.248748
Minibatch accuracy: 78.9%
Validation accuracy: 61.5%
Minibatch loss at step 2500: 12.352250
Minibatch accuracy: 73.4%
Validation accuracy: 61.1%
Minibatch loss at step 3000: 8.275083
Minibatch accuracy: 79.7%
Validation accuracy: 61.0%
Test accuracy: 87.5%


In [12]:
num_steps = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 466.143433
Minibatch accuracy: 6.2%
Validation accuracy: 20.4%
Test accuracy: 86.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


More Steps

In [13]:
num_steps = [3001, 5001, 7001, 10001]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for steps in num_steps:
        for step in range(steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
   
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        print("Number of Steps: %s, Test accuracy: %.1f%%" % (steps, accuracy(test_prediction.eval(), test_labels)))

Initialized
Number of Steps: 3001, Test accuracy: 87.6%
Number of Steps: 5001, Test accuracy: 87.6%
Number of Steps: 7001, Test accuracy: 90.3%
Number of Steps: 10001, Test accuracy: 91.5%


Decrease beta

In [14]:
beta = [0.1, 0.01, 0.005, 0.001, 0.0005, 0.0001]

batch_size = 128
step = 10001

graph = tf.Graph()
with graph.as_default():
    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    #Hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    #Add droupout
    hidden_layer_drop = tf.nn.dropout(hidden_layer,0.5)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    for b in beta:

        #Training computation
        logits = tf.matmul(hidden_layer_drop, weights) + biases
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        new_loss = loss + b*tf.nn.l2_loss(weights)
  
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
        valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(valid_relu, weights)+ biases))
        test_relu = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
        test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(test_relu, weights)+ biases))
        
        with tf.Session(graph=graph) as session:
            tf.initialize_all_variables().run()
            print("Initialized")

            for step in range(steps):
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
   
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]

                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            print("Beta: %s, Test accuracy: %.1f%%" % (b, accuracy(test_prediction.eval(), test_labels)))


Initialized
Beta: 0.1, Test accuracy: 85.5%
Initialized
Beta: 0.01, Test accuracy: 88.9%
Initialized
Beta: 0.005, Test accuracy: 89.1%
Initialized
Beta: 0.001, Test accuracy: 89.5%
Initialized
Beta: 0.0005, Test accuracy: 89.4%
Initialized
Beta: 0.0001, Test accuracy: 89.4%


Optimize he probability that each element is kept in the dropout (keep_prob parameter)

In [15]:
keep_prob = [0.9, 0.7, 0.6, 0.5, 0.4, 0.3]

beta = 0.0001
batch_size = 128
step = 10001

graph = tf.Graph()
with graph.as_default():
    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    #Hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)

    
    for prob in keep_prob:
        
        #Add droupout
        hidden_layer_drop = tf.nn.dropout(hidden_layer,prob)
    
        #Variables
        weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        #Training computation
        logits = tf.matmul(hidden_layer_drop, weights) + biases
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        new_loss = loss + beta*tf.nn.l2_loss(weights)
  
        # Optimizer.
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(new_loss)
  
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)
        valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
        valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(valid_relu, weights)+ biases))
        test_relu = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
        test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(test_relu, weights)+ biases))
        
        with tf.Session(graph=graph) as session:
            tf.initialize_all_variables().run()
            print("Initialized")

            for step in range(steps):
                offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
   
                batch_data = train_dataset[offset:(offset + batch_size), :]
                batch_labels = train_labels[offset:(offset + batch_size), :]

                feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            print("keep_prob: %s, Test accuracy: %.1f%%" % (prob, accuracy(test_prediction.eval(), test_labels)))


Initialized
keep_prob: 0.9, Test accuracy: 91.3%
Initialized
keep_prob: 0.7, Test accuracy: 90.6%
Initialized
keep_prob: 0.6, Test accuracy: 89.5%
Initialized
keep_prob: 0.5, Test accuracy: 89.4%
Initialized
keep_prob: 0.4, Test accuracy: 89.0%
Initialized
keep_prob: 0.3, Test accuracy: 86.7%


Learning Rate Decay

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    

    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    #Hidden layer
    hidden_nodes = 1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    #Add droupout
    hidden_layer_drop = tf.nn.dropout(hidden_layer,0.5)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    beta = 0.0001
    logits = tf.matmul(hidden_layer_drop, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    new_loss = loss + beta*tf.nn.l2_loss(weights)
  
    # Optimizer.
    global_step = tf.Variable(0)
    starter_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.90)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(new_loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(valid_relu, weights)+ biases))
    test_relu = tf.nn.relu( tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.nn.relu(tf.matmul(test_relu, weights)+ biases))


In [17]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 430.837830
Minibatch accuracy: 10.2%
Validation accuracy: 11.2%
Minibatch loss at step 1000: 62.686111
Minibatch accuracy: 77.3%
Validation accuracy: 65.9%
Minibatch loss at step 2000: 42.035610
Minibatch accuracy: 75.8%
Validation accuracy: 66.6%
Minibatch loss at step 3000: 30.776707
Minibatch accuracy: 82.0%
Validation accuracy: 67.2%
Minibatch loss at step 4000: 39.927654
Minibatch accuracy: 75.0%
Validation accuracy: 67.9%
Minibatch loss at step 5000: 37.656200
Minibatch accuracy: 78.1%
Validation accuracy: 68.2%
Minibatch loss at step 6000: 23.869341
Minibatch accuracy: 83.6%
Validation accuracy: 67.8%
Minibatch loss at step 7000: 26.350317
Minibatch accuracy: 77.3%
Validation accuracy: 68.3%
Minibatch loss at step 8000: 26.762880
Minibatch accuracy: 79.7%
Validation accuracy: 68.4%
Minibatch loss at step 9000: 27.016722
Minibatch accuracy: 80.5%
Validation accuracy: 68.8%
Minibatch loss at step 10000: 16.931696
Minibatch accuracy: 81.2%
Vali

More Layers: 2-hidden layer neural network

In [68]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 1024
beta = 0.0001

graph = tf.Graph()
with graph.as_default():
    

    #Input Data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Hidden layer
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1], stddev=np.sqrt(2.0 / hidden_nodes1)))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    
    #2nd hidden layer
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], stddev=np.sqrt(2.0 / hidden_nodes2)))
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer, hidden_weights2) + hidden_biases2)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(hidden_layer2, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    new_loss = loss + beta*tf.nn.l2_loss(weights)
   
    # Optimizer.
    global_step = tf.Variable(0)
    starter_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.90, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(new_loss, global_step=global_step)
  
    # Predictions for the training, validation, and test data.
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_relu1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_relu2 = tf.nn.relu(tf.matmul(valid_relu1, hidden_weights2) + hidden_biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu2, weights) + biases)
    
    test_relu1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_relu2 = tf.nn.relu(tf.matmul(test_relu1, hidden_weights2) + hidden_biases2)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu2, weights) + biases)
    

In [69]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 11.754388
Minibatch accuracy: 4.7%
Validation accuracy: 23.7%
Minibatch loss at step 1000: 0.476968
Minibatch accuracy: 88.3%
Validation accuracy: 69.1%
Minibatch loss at step 2000: 0.448859
Minibatch accuracy: 85.2%
Validation accuracy: 71.2%
Minibatch loss at step 3000: 0.247929
Minibatch accuracy: 94.5%
Validation accuracy: 72.4%
Minibatch loss at step 4000: 0.225168
Minibatch accuracy: 94.5%
Validation accuracy: 73.0%
Minibatch loss at step 5000: 0.363310
Minibatch accuracy: 88.3%
Validation accuracy: 72.4%
Minibatch loss at step 6000: 0.231724
Minibatch accuracy: 96.1%
Validation accuracy: 73.2%
Minibatch loss at step 7000: 0.183522
Minibatch accuracy: 96.9%
Validation accuracy: 73.9%
Minibatch loss at step 8000: 0.152806
Minibatch accuracy: 96.9%
Validation accuracy: 73.8%
Minibatch loss at step 9000: 0.182612
Minibatch accuracy: 96.1%
Validation accuracy: 74.0%
Minibatch loss at step 10000: 0.131170
Minibatch accuracy: 96.9%
Validation accur